# Lab 3: Build a Search Agent

In this lab, we'll use the Azure AI Agent Service to create an agent that is able to retrieve information from documents stored in Azure AI Search, a vector database. This pattern is known as retrieval augmented generation or RAG. The documents that we'll be searching are health insurance policies.

#### Step 1: Load packages

In [10]:
# %pip install azure-ai-agents --quiet

import os
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

load_dotenv() # Loads the environment variables and credentials we need to setup the agent

print("✅ Packages loaded successfully!")
print("✅ Ready to configure Azure AI Search")

✅ Packages loaded successfully!
✅ Ready to configure Azure AI Search


#### Step 2: Connect to your Azure AI Foundry project

In [11]:
# Connecting to our Azure AI Foundry project, which will allow us to use the deployed gpt-4o model
endpoint = os.environ.get("AIPROJECT_ENDPOINT", "https://ai-foundry-demo1.services.ai.azure.com/api/projects/firstProject")
model = os.getenv("CHAT_MODEL")

project_client = AIProjectClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential()
)

#### Step 3: Connect to your Azure AI Search index

In [ ]:
# Get the connections from your project
conn_list = project_client.connections.list()
search_connection = None

print("Available connections:")
for conn in conn_list:
    # Print connection details for debugging
    conn_type = getattr(conn, 'connection_type', None) or getattr(conn, 'type', None)
    conn_name = getattr(conn, 'name', 'Unknown')
    print(f"  - {conn_name}: {conn_type}")
    
    # Look for Azure AI Search connection
    if conn_type and ("CognitiveSearch" in str(conn_type) or "Search" in str(conn_type)):
        search_connection = conn
        print(f"✅ Found Azure AI Search connection: {conn_name} (ID: {conn.id})")

if search_connection:
    # Store the connection details for use in agent creation
    search_connection_id = search_connection.id
    search_index_name = "health-plan"  # The index name containing health plan documents
    
    print(f"✅ Using connection ID: {search_connection_id}")
    print(f"✅ Target index: {search_index_name}")
    has_search_connection = True
else:
    print("⚠️  No Azure AI Search connection found!")
    print("📝 Note: We'll create a general health plan agent without search indexing.")
    print("   In a production setup, you would configure Azure AI Search with health plan documents.")
    has_search_connection = False

print(f"\n🔧 Configuration complete. Search connection available: {has_search_connection}")

Available connections:
  - demoazureopenai1223: ConnectionType.AZURE_OPEN_AI
⚠️  No Azure AI Search connection found!
Available connection types:
  - demoazureopenai1223: ConnectionType.AZURE_OPEN_AI
⚠️  No Azure AI Search connection found!
Available connection types:


#### Step 4: Define the search agent

In [ ]:
# Create the search agent
if has_search_connection:
    # Create agent with search capabilities
    search_agent = project_client.agents.create_agent(
        model=model,
        name="search-agent",
        instructions="""You are a helpful agent that is an expert at searching health plan documents. 
        You have access to health insurance plan documents and can answer questions about coverage, 
        benefits, deductibles, and other plan details. Always provide specific information from the 
        documents when available.""",
        tools=[{"type": "file_search"}],
    )
    print(f"✅ Search-enabled agent created successfully!")
else:
    # Create a general health plan agent without search
    search_agent = project_client.agents.create_agent(
        model=model,
        name="health-plan-agent",
        instructions="""You are a helpful agent that is an expert in health insurance plans. 
        You can provide general information about health insurance concepts like deductibles, 
        copays, coverage types, and plan structures. When asked about specific plans like 
        'Northwind Standard', provide realistic examples of what such a plan might include 
        based on typical health insurance offerings.""",
    )
    print(f"✅ General health plan agent created successfully!")

print(f"   Agent ID: {search_agent.id}")
print(f"   Model: {model}")
print(f"   Type: {'Search-enabled' if has_search_connection else 'General knowledge'}")

#### Step 5: Chat with the search agent

Let's test our search agent by asking it to give us information about the Northwind Standard health plan.

In [ ]:
# The name of the health plan we want to search for
plan_name = 'Northwind Standard'

# Create a thread which is a conversation session between an agent and a user. 
thread = project_client.agents.threads.create()

# Create a message in the thread with the user asking for information about a specific health plan
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"Tell me about the {plan_name} plan.", # The user's message
)

# Run the agent to process the message in the thread
run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=search_agent.id)

# Check if the run was successful
if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    print(f"✅ Run completed with status: {run.status}")

# Fetch all the messages from the thread
messages = project_client.agents.messages.list(thread_id=thread.id)

# Get the agent's response
agent_response = None
for message in messages:
    if message.role == "assistant":
        if hasattr(message, 'content') and message.content:
            for content_item in message.content:
                if hasattr(content_item, 'text') and content_item.text:
                    agent_response = content_item.text.value
                    break
        break

# Display the agent's response
if agent_response:
    print('\n📋 Agent Response:')
    print(agent_response)
else:
    print("⚠️  No response received from the agent")

# Delete the agent when it's done running
project_client.agents.delete_agent(search_agent.id)
print("\n✅ Agent deleted successfully")